In [ ]:
from pystac_client import Client
from odc.stac import load

In [ ]:
# latlon_min = (-18.06231, 177.71210)
# latlon_max = (-17.54912, 178.25661)

# bbox = [latlon_min[1], latlon_min[0], latlon_max[1], latlon_max[0]]

# bbox over viti levu
bbox = [177.71210, -18.06231, 178.25661, -17.54912]

In [ ]:
catalog = "https://stac.staging.digitalearthpacific.io/"
client = Client.open(catalog)

In [ ]:
items = client.search(collections="dep_s2_geomad", bbox=bbox).item_collection()

print(f"Found {len(items)} items")

In [ ]:
data = load(items, bbox=bbox, chunks={"x": 2048, "y": 2048})
data

In [ ]:
bands = ["bcmad", "smad", "emad"]

data.squeeze().emad.plot.imshow(robust=True)

In [ ]:
data.emad.plot.hist(bins=100)

In [ ]:
print(data.emad.compute().quantile([0.05, 0.95]).values)

In [ ]:
import rasterio

with rasterio.Env(AWS_NO_SIGN_REQUEST="YES"):
    loaded = data[["red", "green", "blue", "smad", "bcmad", "emad"]].compute()

In [ ]:
for var in ["smad", "bcmad", "emad"]:
    # print min, max, and 10/90 percentiles
    print(
        f"{var}: {loaded[var].min().values}, {loaded[var].max().values}, {loaded[var].quantile([0.1, 0.9]).values}"
    )

In [ ]:
loaded.emad.plot.hist(bins=100)

In [ ]:
import folium

m = folium.Map(location=loaded.odc.geobox.extent.centroid.to_crs("epsg:4326").coords[0][::-1], zoom_start=10)

loaded.odc.to_rgba(vmin=1000, vmax=3000).odc.add_to(m)

for band in ["smad", "emad", "bcmad"]:
    loaded[band].odc.add_to(m, robust=True)

folium.LayerControl().add_to(m)

m

In [ ]:
# m.save("index.html")